### 1. Data Preprocessing (Cleaning Reviews)

In [ ]:
import pandas as pd
import numpy as np
import re
import langid

df = pd.read_csv("booking_reviews_combined.csv", encoding="utf-8")
print(f"Number of reviews before data preprocessing: {len(df)}")

# --------------------------
# 1. Detect language
# --------------------------
# Fill N/A
df['review_title'] = df['review_title'].fillna('')
df['review_positive_text'] = df['review_positive_text'].fillna('')
df['review_negative_text'] = df['review_negative_text'].fillna('')

# Detect language
df['detected_lang_title'] = df['review_title'].apply(lambda x: langid.classify(str(x))[0] if x.strip() else np.nan)
df['detected_lang_pos'] = df['review_positive_text'].apply(lambda x: langid.classify(str(x))[0] if x.strip() else np.nan)
df['detected_lang_neg'] = df['review_negative_text'].apply(lambda x: langid.classify(str(x))[0] if x.strip() else np.nan)

file_path = 'booking_reviews_detected_lang.csv'
df.to_csv(file_path, index=False, encoding="utf-8")
print(f"Translated reviews saved to {file_path}.")


In [ ]:
# --------------------------
# 0. Load data and initialize
# --------------------------
file_path = 'booking_reviews_detected_lang.csv'
df = pd.read_csv(file_path, encoding="utf-8")
df["removal_reason"] = None
total_rows = len(df)
print(f"Total number of reviews before cleaning: {total_rows}")

# Backup original text before cleaning
original_text_cols = df[["review_positive_text", "review_negative_text"]].copy()

# --------------------------
# 1. Remove foreign-language reviews
# --------------------------
foreign_lang_mask = (
    ((df['detected_lang_pos'].notna() & (df['detected_lang_pos'] != 'en')) &
     (df['detected_lang_neg'].notna() & (df['detected_lang_neg'] != 'en')))
    |
    ((df['detected_lang_pos'].notna() & (df['detected_lang_pos'] != 'en')) &
     (df['detected_lang_neg'].isna()))
    |
    ((df['detected_lang_neg'].notna() & (df['detected_lang_neg'] != 'en')) &
     (df['detected_lang_pos'].isna()))
)
df.loc[foreign_lang_mask, "removal_reason"] = "Foreign language"
num_foreign = foreign_lang_mask.sum()
print(f"Number of foreign language reviews: {num_foreign}, making up {num_foreign/total_rows:.2%} of the original data")

# --------------------------
# 2. Define text cleaning functions
# --------------------------
def clean_text(text):
    if not isinstance(text, str):
        return np.nan
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", "", text)
    text = re.sub(r"\.\s+", ". ", text)
    return text.strip() if text.strip() else np.nan

GENERIC_WORDS = {
    # Positive
    "good", "very good", "all good", "all ok", "nice", "excellent", "enjoyable", "great", "ok", "fine",
    "decent", "amazing", "awesome", "perfect", "super", "satisfactory", "satisfying",
    "wonderful", "fantastic", "best", "cool", "superb", "nice place", "happy", "alright",
    # Negative
    "nothing", "none", "no", "n/a", "na", "nil", "no comments", "no complaint", "no issues",
    "not bad", "not good", "neutral", "meh", "so so", "average", "passable", "okay", "fair"
}

def clean_reviews(df, cols, generic_words, min_words=5):
    pattern = "|".join(map(re.escape, generic_words))

    for col in cols:
        df[col] = df[col].apply(clean_text)

        is_generic = df[col].str.lower().str.fullmatch(pattern, na=False)
        df.loc[is_generic, col] = np.nan

        is_short = df[col].str.split().str.len() < min_words
        df.loc[is_short, col] = np.nan

    return df

# --------------------------
# 3. Clean short/generic reviews
# --------------------------
df = clean_reviews(df, ["review_positive_text", "review_negative_text"], GENERIC_WORDS)

# --------------------------
# 4. Remove rows with no valid text
# --------------------------
empty_mask = df['review_positive_text'].isna() & df['review_negative_text'].isna()
df.loc[empty_mask & df["removal_reason"].isna(), "removal_reason"] = "No valid text"

# --------------------------
# 5. Split cleaned vs removed
# --------------------------
df_cleaned = df[df["removal_reason"].isna()].copy()
removed_rows = df[df["removal_reason"].notna()].copy()

# Restore original review text in removed rows
removed_rows[["review_positive_text", "review_negative_text"]] = original_text_cols.loc[removed_rows.index]

# --------------------------
# 6. Save outputs and print summary
# --------------------------
df_cleaned.to_csv("booking_reviews_clean.csv", index=False)
removed_rows.to_csv("booking_reviews_filtered_out.csv", index=False)

pct_kept = len(df_cleaned) / total_rows
pct_removed = len(removed_rows) / total_rows

print(f"\n✅ Cleaned data saved: {len(df_cleaned)} rows ({pct_kept:.2%} of original)")
print(f"🗑️ Removed rows saved: {len(removed_rows)} rows ({pct_removed:.2%} of original)")


Total number of reviews before cleaning: 292507
Number of foreign language reviews: 78710, making up 26.91% of the original data

✅ Cleaned data saved: 185125 rows (63.29% of original)
🗑️ Removed rows saved: 107382 rows (36.71% of original)


### 2. Chunking Reviews

In [ ]:
import pandas as pd
import numpy as np
import re
import ast
from transformers import pipeline, AutoTokenizer

# Load dataset
booking_reviews = pd.read_csv('booking_reviews_clean.csv', dtype={'hotel_id':str})
booking_reviews.head()

,review_ID,reviewer_name,reviewer_country,review_room_name,review_num_nights,review_stay_date,review_traveler_type,review_score,review_title,review_positive_text,review_negative_text,hotel_name,detected_lang_title,detected_lang_pos,detected_lang_neg,removal_reason
0,1,Gabriela,United States,"Deluxe Twin, Window",2,Jan-25,Family,10.0,Comfortable and convenient,This was a last minute booking so we arrived t...,NaN,A Hotel Joo Chiat,en,en,NaN,NaN
1,2,A,Singapore,"Superior Double, No Window",4,Feb-25,Couple,7.0,Overall I'm quite happy with the stay except f...,NaN,A little costly for a budget hotel.,A Hotel Joo Chiat,en,en,en,NaN
2,3,Sumarni,Australia,"Superior Twin, No Window",3,Feb-25,Group,8.0,Good and achieve my expectation,NaN,The fire alarm and no communication with the p...,A Hotel Joo Chiat,en,en,en,NaN
3,4,Samsul,Australia,"Deluxe Double, Window",6,Feb-25,Couple,8.0,Need room makeup… every stay..,we are regularlythe hotel need to overhaul\nth...,Paid extra for little bit big View.. with Win...,A Hotel Joo Chiat,en,en,en,NaN
4,5,Mark,Malaysia,"Superior Twin, No Window",1,Nov-24,Solo traveller,8.0,CH,"Good location, nearby shops and walking distan...","Room light, not bright enough.",A Hotel Joo Chiat,en,en,en,NaN


In [ ]:
# Set up sentiment pipeline and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm


# For sentiment analysis (must match DistilBERT model)
sentiment_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

# For token-based chunking (matches embedding model)
chunking_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
MAX_TOKENS = 150

# Sentence tokenizer fallback (regex-based)
def simple_sentence_tokenizer(text):
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', str(text)) if s.strip()]

# Dynamic chunker based on token length
def chunk_sentences_by_token_limit(sentences, max_tokens=MAX_TOKENS):
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = len(chunking_tokenizer.encode(sentence, add_special_tokens=False))

        if current_tokens + sentence_tokens > max_tokens and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_tokens = sentence_tokens
        else:
            current_chunk.append(sentence)
            current_tokens += sentence_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


# Function to process each review row
def chunk_review_with_sentiment_title(row):
    review_id = row['review_ID']

    title_text = str(row['review_title']) if pd.notnull(row['review_title']) else ""
    positive_text = str(row['review_positive_text']) if pd.notnull(row['review_positive_text']) else ""
    negative_text = str(row['review_negative_text']) if pd.notnull(row['review_negative_text']) else ""

    # Classify title sentiment and tokenise
    title_sent = sentiment_pipeline(title_text)[0]['label']
    title_sentences = simple_sentence_tokenizer(title_text)
    positive_sentences = simple_sentence_tokenizer(positive_text)
    negative_sentences = simple_sentence_tokenizer(negative_text)

    # Merge title sentences into sentiment group
    if title_sent == 'POSITIVE':
        positive_sentences = title_sentences + positive_sentences
    else:
        negative_sentences = title_sentences + negative_sentences

    # Group by sentiment using token-based chunking
    chunks = []
    for sentiment, sentences in [('Positive', positive_sentences), ('Negative', negative_sentences)]:
        grouped_chunks = chunk_sentences_by_token_limit(sentences, max_tokens=MAX_TOKENS)
        for i, chunk in enumerate(grouped_chunks):
            chunks.append({
                'review_id': review_id,
                'chunk_id': f"{review_id}_{sentiment}_{i}",
                'hotel_name': row['hotel_name'],
                'review_score': row['review_score'],
                'sentiment': sentiment,
                'chunk_text': chunk,
                'reviewer_name': row['reviewer_name'],
                'reviewer_country': row['reviewer_country'],
                'review_room_name': row['review_room_name'],
                'review_num_nights': row['review_num_nights'],
                'review_stay_date': row['review_stay_date'],
                'review_traveler_type': row['review_traveler_type']
            })

    return chunks

# Apply to your DataFrame
processed_chunks = []
for _, row in tqdm(booking_reviews.iterrows(), total=len(booking_reviews), desc="Processing reviews"):
    processed_chunks.extend(chunk_review_with_sentiment_title(row))

# Convert to DataFrame and export
chunks_df = pd.DataFrame(processed_chunks)
chunks_df.to_csv("review_chunks_with_sentiment.csv", index=False)
chunks_df.head(10)


In [ ]:
# --------------------------
# Remove rows with short reviews
# --------------------------

# Load dataset
review_chunks_with_sentiment = pd.read_csv('review_chunks_with_sentiment.csv')

# Filter out rows with short reviews (less than or equal to 3 words)
filtered_df = review_chunks_with_sentiment[review_chunks_with_sentiment['chunk_text'].str.split().apply(len) <= 3]
print(f"Number of rows in filtered_df: {len(filtered_df)}")
print(f"Number of unique values: {len(filtered_df['chunk_text'].unique())}")

# Save clean file
review_chunks_with_sentiment_clean = review_chunks_with_sentiment.drop(filtered_df.index)
review_chunks_with_sentiment_clean.to_csv("review_chunks_with_sentiment_clean.csv", index=False)

print(f"\nNumber of rows in original review_chunks_with_sentiment: {len(review_chunks_with_sentiment)}")
print(f"Number of rows after filtering: {len(review_chunks_with_sentiment_clean)}")

print("Dataset saved to review_chunks_with_sentiment_clean.csv")

### 3. Prep for Weaviate

In [ ]:
import pandas as pd

# ---- Flatten Review Scores ----
def stringify_review_scores(scores):
    if pd.isna(scores): return ""
    if isinstance(scores, str):
        try: scores = ast.literal_eval(scores)
        except: return str(scores)
    return ". ".join([f"{s['category']}: {s['score']}" for s in scores])


# ---- Flatten Cot / Extra Bed Policies ----
def stringify_bed_policies_safe(policies):
    if pd.isna(policies): return ""
    try:
        if isinstance(policies, str):
            policies = ast.literal_eval(policies)
        if not isinstance(policies, dict):
            return str(policies)
    except:
        return str(policies)

    flat = []
    for age, options in policies.items():
        if isinstance(options, dict):
            details = ', '.join([f"{k} - {v}" for k, v in options.items()])
            flat.append(f"{age}: {details}")
        else:
            flat.append(f"{age}: {options}")
    return ". ".join(flat)


# ---- Flatten Payment Methods ----
def stringify_payment_methods(pm):
    if pd.isna(pm): return ""
    if isinstance(pm, str):
        try: pm = ast.literal_eval(pm)
        except: return str(pm)
    cards = ", ".join(pm.get("accepted_cards", []))
    cash = pm.get("cash_accepted", "Unknown")
    return f"Accepted Cards: {cards}. Cash Accepted: {cash}"


# ---- Flatten Room Details ----
def stringify_room_details(rooms):
    if pd.isna(rooms): return ""
    if isinstance(rooms, str):
        try: rooms = ast.literal_eval(rooms)
        except: return str(rooms)
    summary = []
    for r in rooms:
        desc = f"Room: {r.get('Room Name')}, Size: {r.get('Room Size')}, Features: {r.get('Room Highlights & Facilities', '')}"
        summary.append(desc)
    return ". ".join(summary)


# ---- Flatten Surroundings ----
def stringify_surroundings(surr):
    if pd.isna(surr): return ""
    if isinstance(surr, str):
        try: surr = ast.literal_eval(surr)
        except: return str(surr)
    flat = []
    for cat, items in surr.items():
        entries = ", ".join([f"{i['name']} ({i.get('distance', 'n/a')})" for i in items if isinstance(i, dict) and 'name' in i])
        flat.append(f"{cat}: {entries}")
    return ". ".join(flat)

# ---- Flatten Facilities ----
def stringify_facilities(facilities):
    if pd.isna(facilities): return ""
    if isinstance(facilities, str):
        try: facilities = ast.literal_eval(facilities)
        except: return str(facilities)
    flat = []
    for cat, items in facilities.items():
        flat.append(f"{cat}: {', '.join(items)}")
    return ". ".join(flat)

In [ ]:
property_df = pd.read_csv('property_df.csv')
review_chunks = pd.read_csv('review_chunks_with_sentiment_clean.csv')

# Working on a copy of property_data
property_df_flat = property_df.copy()

# ---- Apply functions ----
property_df_flat["review_scores_text"] = property_df_flat["review_scores"].apply(stringify_review_scores)
property_df_flat["cot_extra_bed_policies_text"] = property_df_flat["cot_extra_bed_policies"].apply(stringify_bed_policies_safe)
property_df_flat["payment_methods_text"] = property_df_flat["payment_methods"].apply(stringify_payment_methods)
property_df_flat["room_details_text"] = property_df_flat["room_details"].apply(stringify_room_details)
property_df_flat["surroundings_text"] = property_df_flat["surroundings"].apply(stringify_surroundings)
property_df_flat["facilities_text"] = property_df_flat["facilities"].apply(stringify_facilities)

# Ensure num_reviews is an integer (not float)
property_df_flat["num_reviews"] = property_df_flat["num_reviews"].fillna(0).astype(int)

# Drop original cols
property_df_flat = property_df_flat.drop(
    columns=["review_scores", "cot_extra_bed_policies", "payment_methods", "room_details", "surroundings", "facilities"]
)

property_df_flat.to_csv("property_df_flat.csv", index=False)
print("Property data saved to property_df_flat.csv")

Property data saved to property_df_flat.csv


### 4. Compute Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# Load your CSV
df = pd.read_csv("review_chunks_with_sentiment_clean.csv")

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings
df['embedding'] = list(tqdm(model.encode(df['chunk_text'].tolist(), batch_size=64, show_progress_bar=True)))


Batches:   0%|          | 0/4583 [00:00<?, ?it/s]

100%|██████████| 293302/293302 [00:00<00:00, 1301982.56it/s]


In [ ]:
# Save the DataFrame with embeddings to a new CSV file
df.to_csv("review_chunks_with_sentiment_embeddings.csv", index=False)
print("Embeddings saved to review_chunks_with_sentiment_embeddings.csv")

Embeddings saved to review_chunks_with_sentiment_embeddings.csv
